In [6]:
import asyncio
from pyzeebe import ZeebeWorker, Job, create_camunda_cloud_channel, ZeebeClient
from datetime import date
import requests
import pycountry_convert

In [7]:
ClusterURL = "fc43de37-01a5-41a0-a9d8-51d621169f35.dsm-1.zeebe.camunda.io:443"
ClientID = "UPPG5KY-b6ZRN1Yf_iuXG87hucOa286k"
ClientSecret = "J~pZPr6LZ6NxBa2Qwr7DVd8YwZahW35v8RIAMsi~h~VThu0.76GzL73sLB37CR2r"
ClusterID = "fc43de37-01a5-41a0-a9d8-51d621169f35"
Region = "dsm-1"

In [8]:
channel = create_camunda_cloud_channel(client_id=ClientID, 
                                       client_secret=ClientSecret, 
                                       cluster_id=ClusterID, 
                                       region=Region)

In [9]:
# Create a zeebe client and worker
client = ZeebeClient(channel)
worker = ZeebeWorker(channel)

In [10]:
process_key = await client.run_process(bpmn_process_id='KundeProzess', variables={})

In [11]:
# Liste der Personen, die versichert werden sollen
personen = []

@worker.task(task_type="AntragVersendenType")
async def AntragVersenden(RZiel: str, RbJahr: float, RbMonat: float, RbTag: float, ReJahr: float, ReMonat: float, ReTag: float,
    GesK: float, AnzPers: float, Corona: bool, Reiseruecktritt: bool, PNr: str, VName: str, NName: str,
    GebJ: float, GebM: float,GebT: float, TelN: str, EMail: str, Strasse: str, HausN: str, Plz: str, Ort: str, LCode: str):

    # Weitergabe der Eingabedaten von VN an Versicherung
    corr_key_a="Message_1l1hl4j"
    await client.publish_message(name=corr_key_a, correlation_key=corr_key_a, variables={"RZiel": RZiel, "RbJahr": RbJahr,
    "RbMonat": RbMonat, "RbTag": RbTag, "ReJahr": ReJahr, "ReMonat": ReMonat, "ReTag": ReTag, "GesK": GesK, "AnzPers": AnzPers,
    "Corona": Corona, "Reiseruecktritt": Reiseruecktritt, "PNr": PNr, "VName": VName, "NName": NName, "GebJ": GebJ, "GebM": GebM,
    "GebT": GebT, "TelN": TelN, "EMail": EMail, "Strasse": Strasse, "HausN": HausN, "Plz": Plz, "Ort": Ort, "LCode": LCode})
    
    return {}

@worker.task(task_type="AntragErnVersendenType")
async def AntragErnVersenden(RZiel: str, RbJahr: float, RbMonat: float, RbTag: float, ReJahr: float, ReMonat: float, ReTag: float,
    GesK: float, AnzPers: float, Corona: bool, Reiseruecktritt: bool, PNr: str, VName: str, NName: str,
    GebJ: float, GebM: float, GebT: float, TelN: str, EMail: str, Strasse: str, HausN: str, Plz: str, Ort: str, LCode: str):

    # Weitergabe der vervollständigten Eingabedaten von VN an Versicherung
    corr_key_b="Message_3cnlpm8"
    await client.publish_message(name=corr_key_b, correlation_key=corr_key_b, variables={"RZiel": RZiel, "RbJahr": RbJahr,
    "RbMonat": RbMonat, "RbTag": RbTag, "ReJahr": ReJahr, "ReMonat": ReMonat, "ReTag": ReTag, "GesK": GesK, "AnzPers": AnzPers,
    "Corona": Corona, "Reiseruecktritt": Reiseruecktritt, "PNr": PNr, "VName": VName, "NName": NName, "GebJ": GebJ, "GebM": GebM,
    "GebT": GebT, "TelN": TelN, "EMail": EMail, "Strasse": Strasse, "HausN": HausN, "Plz": Plz, "Ort": Ort, "LCode": LCode})
    
    return {}

@worker.task(task_type="AnfrDatenType")
async def AnfrDaten(Message: str):

    # Versicherung bittet VN, Eingabedaten zu vervollständigen
    corr_key_c="Message_1knatbt"
    await client.publish_message(name=corr_key_c, correlation_key=corr_key_c, variables={"Message": Message})
    
    return {}

@worker.task(task_type="AntrAblDatenType")
async def AnfrAblDaten(Message: str):

    # Versicherung informiert VN, dass Antrag wegen falscher Daten (<=18 Jahre oder Wohnsitz nicht in DEU) abgelehnt
    corr_key_d="Message_25917sj"
    await client.publish_message(name=corr_key_d, correlation_key=corr_key_d, variables={"Message": Message})
    
    return {}

# Versicherung überprüft Eingabedaten des VN
@worker.task(task_type="AntragPruefenType")
async def AntragPruefen(RbJahr: float, RbMonat: float, RbTag: float, 
    ReJahr: float, ReMonat: float, ReTag: float, LCode: str, GebJ: float,
    GebM: float, GebT: float):

    rb = date(RbJahr, RbMonat, RbTag)
    re = date(ReJahr, ReMonat, ReTag)
    geb = date(GebJ, GebM, GebT)
    today = date.today()
    alter = today.year - GebJ - ((today.month, today.day) < (GebM, GebT)) 

    # Reiseende darf nicht vor Reisebeginn liegen
    if re < rb:
        return {"Ergebnis": -1, "Message": "Reiseende vor Reisebeginn"}

    # Reisebeginn darf nicht in Vergangenheit liegen
    if rb < today:
        return {"Ergebnis": -1, "Message": "Reisebeginn in der Vergangenheit"}
    
    # VN muss aus Deutschland kommen
    if LCode != "DEU":
        return {"Ergebnis": 0, "Message": "VN mit Wohnsitz ausserhalb Deutschlands"}
    
    # VN muss mindestens 18 Jahre alt sein
    if alter < 18:
        return {"Ergebnis": 0, "Message": "VN ist jünger als 18 Jahre"}

    
    return {"Ergebnis": 1, "Message": ""}


@worker.task(task_type="RwPruefenType")
async def RwPruefen(RZiel: str):

    # JSON-Datei von URL abrufen und in tmp abspeichern
    api_url = "https://www.auswaertiges-amt.de/opendata/travelwarning"
    response = requests.get(api_url)
    tmp = response.json()['response']

    # Reisewarnung des Reiseziels herausfinden und zurückgeben
    for i in range(199000,299000):
        try:
            if tmp[str(i)]['iso3CountryCode'] == RZiel:
                return {"Warnung": tmp[str(i)]['warning']}
        except KeyError:
            pass

    return {"Warnung": False}

@worker.task(task_type="AntrRwAblType")
async def AntrRwAbl(Ort: str):

    # Versicherung informiert VN, dass Antrag wegen Reisewarnung in Reiseziel abgelehnt
    corr_key_d="Message_25917sj"
    await client.publish_message(name=corr_key_d, correlation_key=corr_key_d, variables={"Message": "abgelehnt, da Reisewarnung vorhanden"})
    
    return {}

@worker.task(task_type="PDAnfordernType")
async def PDAnfordern(Ort: str):

    # Versicherung bittet VN, Personendaten einer weiteren Person, die versichert werden soll, einzugeben
    corr_key_e="Message_287emmg"
    await client.publish_message(name=corr_key_e, correlation_key=corr_key_e, variables={"Message": "Bitte Personendaten angeben"})
    
    return {}

@worker.task(task_type="PDVersendenType")
async def PDVersenden(VName2: str, NName2: str, GebJ2: float, GebM2: float, GebT2: float, Kind: bool):

    # VN übermittelt geforderte Personendaten an Versicherung
    corr_key_f="Message_33t7hta"
    await client.publish_message(name=corr_key_f, correlation_key=corr_key_f, variables={"VName2": VName2, "NName2": NName2, 
    "GebJ2": GebJ2, "GebM2": GebM2, "GebT2": GebT2, "Kind": Kind})
    
    return {}

@worker.task(task_type="PDSpeichernType")
async def PDSpeichern(VName2: str, NName2: str, GebJ2: float, GebM2: float, GebT2: float, Kind: bool, AnzPers: float):
    
    Geb2 = date(GebJ2, GebM2, GebT2)

    # Personenanzahl um 1 reduzieren, da Daten von einer weiteren Person erhalten
    AnzPers = AnzPers - 1

    print(personen)
    
    # Personendaten in Liste personen abspeichern
    personen.append({
      "firstName": VName2,
      "lastName": NName2,
      "birthDate": str(Geb2),
      "childOfInsuranceTaker": Kind
    })

    return {"AnzPers": AnzPers}


@worker.task(task_type="PnSuchenType")
async def PnSuchen(PNr: str):

    # GET-Anfrage an URL (Partner suchen)
    api_url = "https://travel-insurance-api.aws-playground.viadee.cloud/partner/"
    response = requests.get(api_url+PNr, auth=("user2", "6oKwPoFt"))
    
    # auf verschiedene Statuscodes der GET-Anfrage reagieren
    while response.status_code == 500:
        response = requests.get(api_url+PNr, auth=('user2', '6oKwPoFt'))
    if response.status_code == 404:
        return {"PNrGefunden": "Nein", "Neukunde": False}
    if response.status_code == 200:
        return {"PNrGefunden": "Ja", "Neukunde": False}
    
@worker.task(task_type="AdresseSuchenType")
async def AdresseSuchen(PNr: str, VName: str, NName: str, GebJ: float, GebM: float, GebT: float, 
    Strasse: str, HausN: str, Plz: str, Ort: str, LCode: str):

    Geb = date(GebJ, GebM, GebT)

    # JSON-Datei für POST-Anfrage erstellen
    adresse = {
        "partnerId": PNr,
        "firstName": VName,
        "lastName": NName,
        "birthDate": str(Geb),
        "address": {
            "street": Strasse,
            "number": HausN,
            "postalCode": Plz,
            "city": Ort,
            "countryCode": LCode
        }
    }

    # POST-Anfrage an URL (Partner suchen)
    api_url = "https://travel-insurance-api.aws-playground.viadee.cloud/partner/search"
    response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)

    # auf verschiedene Statuscodes der POST-Anfrage reagieren
    while response.status_code == 500:
        response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)

    if response.status_code == 404:
        return {"AdresseGefunden": "Nein", "Neukunde": False}

    if response.status_code == 200:
        return {"AdresseGefunden": "Ja", "Neukunde": False}
    
@worker.task(task_type="PNrVerwendenType")
async def PNrVerwenden(VName: str, NName: str, GebJ: float, GebM: float, GebT: float, 
    Strasse: str, HausN: str, Plz: str, Ort: str, LCode: str):

    Geb = date(GebJ, GebM, GebT)
    
    # JSON-Datei für POST-Anfrage erstellen
    adresse = {
        "partnerId": "",
        "firstName": VName,
        "lastName": NName,
        "birthDate": str(Geb),
        "address": {
            "street": Strasse,
            "number": HausN,
            "postalCode": Plz,
            "city": Ort,
            "countryCode": LCode
        }
    }

    # POST-Anfrage an URL (Partner suchen)
    api_url = "https://travel-insurance-api.aws-playground.viadee.cloud/partner/search"
    response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)
    
    # auf Statuscode 500 der POST-Anfrage reagieren
    while response.status_code == 500:
        response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)

    # Partnernummer des VN aus JSON-Datei auslesen und zurückgeben
    partnerID  = response.json()["partnerId"]
    return {"PNr": partnerID, "Neukunde": False}


@worker.task(task_type="NkAnlegenType")
async def NkAnlegen(VName: str, NName: str, GebJ: float, GebM: float, GebT: float, 
    TelN: str, EMail: str, Strasse: str, HausN: str, Plz: str, Ort: str, LCode: str):

    Geb = date(GebJ, GebM, GebT)
    
    # JSON-Datei für POST-Anfrage erstellen
    adresse = {
        "partnerId": "",
        "firstName": VName,
        "lastName": NName,
        "birthDate": str(Geb),
        "phoneNumber": TelN,
        "emailAddress": EMail,
        "address": {
            "street": Strasse,
            "number": HausN,
            "postalCode": Plz,
            "city": Ort,
            "countryCode": LCode
        }
    }

    # POST-Anfrage an URL (Neukunde anlegen)
    api_url = "https://travel-insurance-api.aws-playground.viadee.cloud/partner"
    response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)
    
    # auf Statuscode 500 der POST-Anfrage reagieren
    while response.status_code == 500:
        response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)

    return {"PNr": response.text, "Neukunde": True}


@worker.task(task_type="AdrUebernehmenType")
async def AdrUebernehmen(VName: str, NName: str, GebJ: float, GebM: float, GebT: float, 
    TelN: str, EMail: str, Strasse: str, HausN: str, Plz: str, Ort: str, LCode: str):

    Geb = date(GebJ, GebM, GebT)
    
    # JSON-Datei für POST-Anfrage erstellen
    adresse = {
        "partnerId": "",
        "firstName": VName,
        "lastName": NName,
        "birthDate": str(Geb),
        "phoneNumber": TelN,
        "emailAddress": EMail,
        "address": {
            "street": Strasse,
            "number": HausN,
            "postalCode": Plz,
            "city": Ort,
            "countryCode": LCode
        }
    }

    # POST-Anfrage an URL (Adresse des VN ändern / Neukunde)
    api_url = "https://travel-insurance-api.aws-playground.viadee.cloud/partner"
    response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)

    # auf Statuscode 500 der POST-Anfrage reagieren
    while response.status_code == 500:
        response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=adresse)

    return {"PNr": response.text, "Neukunde": False}



@worker.task(task_type="SpBerechnenType")
async def SpBerechnen(VName: str, NName: str, GebJ: float, GebM: float, GebT: float, 
    TelN: str, EMail: str, Strasse: str, HausN: str, Plz: str, Ort: str, LCode: str,
    RbJahr: float, RbMonat: float, RbTag: float, ReJahr: float, ReMonat: float, ReTag: float,
    RZiel: str):

    # Reisedauer berechnen
    rb = date(RbJahr, RbMonat, RbTag)
    re = date(ReJahr, ReMonat, ReTag)
    Dauer = (re - rb).days

    # ermitteln, ob Reiseland Deutschland ist oder in Europa oder nicht in Europa liegt
    Reiseziel = ""
    if RZiel == "DEU":
        Reiseziel = "Deutschland"
    else:    
        alpha2 = pycountry_convert.country_alpha3_to_country_alpha2(RZiel)
        kontinent = pycountry_convert.country_alpha2_to_continent_code(alpha2)
        if kontinent == "EU":
            Reiseziel = "Nicht-EU-Ausland"
        else:
            Reiseziel = "EU-Ausland"


    return{"Dauer": Dauer, "Reiseziel": Reiseziel}


@worker.task(task_type="VertragSpeichernType")
async def VertragSpeichern(PNr: str, RbJahr: float, RbMonat: float, RbTag: float, ReJahr: float, ReMonat: float,
    ReTag: float, RZiel: str, GesK: float, Selbstbehalt: float, Reiseruecktritt: bool, Corona: bool):

    rb = date(RbJahr, RbMonat, RbTag)
    re = date(ReJahr, ReMonat, ReTag)
    
    # JSON-Datei für POST-Anfrage erstellen
    vertrag = {
        "insuranceTakerId": PNr,
        "travelDetails": {
            "begin": str(rb),
            "end": str(re),
            "destinationCountryCode": RZiel,
            "totalCost": GesK
        },
        "coverage": {
            "costRetention": Selbstbehalt,
            "withTravelAbortionCoverage": Reiseruecktritt,
            "withTravelExtensionCoverage": Reiseruecktritt,
            "withCoronaCoverage": Corona
        },
        "insuredPersons": personen
    }
    
    # POST-Anfrage an URL (Vertrag speichern)
    api_url = "https://travel-insurance-api.aws-playground.viadee.cloud/insurance-policy"
    response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=vertrag)
    
    # auf Statuscode 500 der POST-Anfrage reagieren (20 Versuche)
    # keine VNr wird zurückgegeben, wenn alle Versuche Statuscode 500 liefern
    count = 0
    while response.status_code == 500:
        if count == 20:
            return {"VNr": ""}
        response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json=vertrag)
        count = count + 1
    
    return {"VNr": response.text}

@worker.task(task_type="SpFehlgeschlagenType")
async def SpFehlgeschlagen(Ort: str):

    # Versicherung informiert VN, dass Vertrag wegen Serverfehler nicht gespeichert werden konnte
    corr_key_g="Message_25917sj"
    await client.publish_message(name=corr_key_g, correlation_key=corr_key_g, variables={"Message": "Serverfehler"})
    
    return {}

@worker.task(task_type="VertragVersType")
async def VertragVers(VNr: str):

    # POST-Anfrage an URL (Vertrag drucken und senden lassen)
    api_url = "https://travel-insurance-api.aws-playground.viadee.cloud/document/print-job/insurance-policy/"+VNr
    response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json={})

    # auf Statuscode 500 der POST-Anfrage reagieren
    while response.status_code == 500:
        response = requests.post(api_url, auth=("user2", "6oKwPoFt"), json={})

    # Versicherung informiert VN, dass Antrag zugestimmt wurde
    corr_key_h="Message_2v8j0rr"
    await client.publish_message(name=corr_key_h, correlation_key=corr_key_h, variables={})

    return {}


        

await worker.work()

Failed to activate jobs from the gateway. Exception: ZeebeBackPressureError(). Retrying in 5 seconds...
Failed to activate jobs from the gateway. Exception: ZeebeBackPressureError(). Retrying in 5 seconds...
Failed to activate jobs from the gateway. Exception: ZeebeBackPressureError(). Retrying in 5 seconds...
